Principal Components

In [1]:
import numpy
from sklearn.datasets import fetch_mldata
from six.moves import urllib

In [2]:
from scipy.io import loadmat
mnist_alternative_url = "https://github.com/amplab/datascience-sp14/raw/master/lab7/mldata/mnist-original.mat"
mnist_path = "./mnist-original.mat"
response = urllib.request.urlopen(mnist_alternative_url)
with open(mnist_path, "wb") as f:
    content = response.read()
    f.write(content)
mnist_raw = loadmat(mnist_path)
mnist = {
    "COL_NAMES": ["label", "data"],
    "DESCR": "mldata.org dataset: mnist-original",
    "data": mnist_raw["data"].T,
    "target": mnist_raw["label"][0]
}
print("Success!")

Success!


In [3]:
X = mnist["data"]

In [4]:
X_centered = X - X.mean(axis = 0)
u, s, v = numpy.linalg.svd(X_centered)
c1 = v.T[:,0]
c2 = v.T[:,1]

Projecting down to d dimensions

In [5]:
w2 = v.T[:,:2]
x2d = X_centered.dot(w2)

In [6]:
from sklearn.decomposition import PCA

In [7]:
pca = PCA(n_components = 2)      #same as above using PCA
x2d = pca.fit_transform(X)

In [8]:
print(pca.explained_variance_ratio_)

[0.09746116 0.07155445]


Choosing right number of dimensions

In [9]:
pca = PCA()
pca.fit(X)
cumsum = numpy.cumsum(pca.explained_variance_ratio_)
d = numpy.argmax(cumsum >= 0.95) + 1      #min. num of dimensions to preserve 95% of training set's variance

In [10]:
pca = PCA(n_components = 0.95)    #ratio of variance to preserve
X_reduced = pca.fit_transform(X)

PCA for Compression

In [11]:
pca = PCA(n_components = 150)      #after preserving 95% of variance, each instance has nearly 150 features
X_mnist_compressed = pca.fit_transform(X)
X_mnist_recovered = pca.inverse_transform(X_mnist_compressed)

Incremental PCA

In [12]:
from sklearn.decomposition import IncrementalPCA

In [13]:
n_batches = 100     #splits MNIST dataset into 100 mini batches
incre_pca = IncrementalPCA(n_components = 150)
for X_batch in numpy.array_split(X, n_batches):
    incre_pca.partial_fit(X_batch)
X_mnist_compressed = incre_pca.transform(X)

Randomized PCA

In [14]:
random_pca = PCA(n_components = 150, svd_solver="randomized")
X_reduced = random_pca.fit_transform(X)

Kernel PCA

In [15]:
from sklearn.decomposition import KernelPCA

In [18]:
ker_pca = KernelPCA(n_components = 2, kernel = "rbf", gamma = 0.04)
X_reduced = ker_pca.fit_transform(X)

MemoryError: 

LLE (Locally Linear Embedding)

In [19]:
from sklearn.manifold import LocallyLinearEmbedding

In [ ]:
lle = LocallyLinearEmbedding(n_components = 2, n_neighbors = 10)
X_reduced = lle.fit_transform(X)